In [41]:
import pandas as pd
import numpy as np
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from preprocess import *
from nltk.tokenize import sent_tokenize
from sets import Set



In [42]:
df = pd.read_csv('debate.csv', encoding = 'iso-8859-1')
print df.head()

   Line   Speaker                                               Text     Date
0     1      Holt  Good evening from Hofstra University in Hempst...  9/26/16
1     2  Audience                                         (APPLAUSE)  9/26/16
2     3   Clinton                               How are you, Donald?  9/26/16
3     4  Audience                                         (APPLAUSE)  9/26/16
4     5      Holt                                  Good luck to you.  9/26/16


In [43]:
df_clinton = df[(df['Speaker'] == 'Clinton')]
print df_clinton.head()
clinton_text = df_clinton['Text'].values.tolist()
print len(clinton_text)

    Line  Speaker                                               Text     Date
2      3  Clinton                               How are you, Donald?  9/26/16
7      8  Clinton  Well, thank you, Lester, and thanks to Hofstra...  9/26/16
8      9  Clinton  I also want to see more companies do profit-sh...  9/26/16
13    14  Clinton  Well, I think that trade is an important issue...  9/26/16
21    22  Clinton  Well, let's stop for a second and remember whe...  9/26/16
243


In [44]:
df_trump = df[(df['Speaker'] == 'Trump')]
print df_trump.head()
trump_text = df_trump['Text'].values.tolist()
print len(trump_text)

    Line Speaker                                               Text     Date
10    11   Trump  Thank you, Lester. Our jobs are fleeing the co...  9/26/16
11    12   Trump  We cannot let it happen. Under my plan, I'll b...  9/26/16
15    16   Trump  Well, for one thing -- and before we start on ...  9/26/16
17    18   Trump  Secretary Clinton and others, politicians, sho...  9/26/16
19    20   Trump  Well, the first thing you do is don't let the ...  9/26/16
355


In [67]:
def sentenceTokenize(input_list):
    set_lines = Set()
    
    for line in input_list:
        sent_tokenize_list = sent_tokenize(line)
        
        '''
        temp = ""
        for tt in sent_tokenize_list:
            if len(temp) < 140:
                temp += tt
            else:
                set_lines.add(temp)
                temp = ""
        '''
        set_lines.update(sent_tokenize_list)
       
    return set_lines

clinton_set = list(sentenceTokenize(clinton_text))
trump_set = list(sentenceTokenize(trump_text))

tweets = clinton_set + trump_set

print len(clinton_set)
print len(trump_set)

1224
1967


In [68]:
dir_path = '/home/shreya/workspace/AdvanceProject/'
#file_out = open(dir_path+'Hillary_thresh_2.txt', 'w')
#file_out1 = open(dir_path+'Trump_thresh_2.txt', 'w')

file_out = open(dir_path+'Hillary_Debate_orig.txt', 'w')
file_out1 = open(dir_path+'Trump_Debate_orig.txt', 'w')

for line in clinton_set:
    file_out.write(line.encode("utf8")+'\n!@#$%\n')

for line in trump_set:
    file_out1.write(line.encode("utf8")+'\n!@#$%\n')
    

In [69]:
vectorizer = TfidfVectorizer()
tfidf_vecs = vectorizer.fit_transform(tweets)

In [70]:
dir_path = '/home/shreya/workspace/AdvanceProject/'
#file_out = open(dir_path+'Hillary_thresh_2.txt', 'w')
#file_out1 = open(dir_path+'Trump_thresh_2.txt', 'w')

file_out = open(dir_path+'Hillary_Debate2.txt', 'w')
file_out1 = open(dir_path+'Trump_Debate2.txt', 'w')

In [71]:
k = 2
con_start_idx = len(clinton_set)

for i in range(0, len(clinton_set)):
#for i in random.sample(range(con_start_idx), 5):
    cosine_similarities = linear_kernel(tfidf_vecs[i:i+1], tfidf_vecs)
    cosine_similarities = cosine_similarities.flatten()
    sorted_idx = np.argsort(cosine_similarities)[::-1]
    
   
        
    topk_pro = []
    topk_con = []

    for idx in sorted_idx:
        if idx < con_start_idx and idx != i:
            if cosine_similarities[idx] >= 0.2:
                tweet_score = str(tweets[idx].encode("utf8"))#+"\t"+str(cosine_similarities[idx])
                topk_pro.append(tweet_score)
                if len(topk_pro) == k:
                    break
            else:
                break

    for idx in sorted_idx:
        if idx >= con_start_idx and idx != i:
            if cosine_similarities[idx] >= 0.2:
                tweet_score = str(tweets[idx].encode("utf8"))#+"\t"+str(cosine_similarities[idx])
                topk_con.append(tweet_score)
                if len(topk_con) == k:
                    break
            else:
                break
        
    for tweet in topk_con:
        file_out.write(str(tweets[i].encode("utf8"))+"\n")
        file_out1.write(str(tweet)+"\n")